# SQL PROJECT

### Installing required libraries for the project

In [1]:
#! pip install PyMySQL`
#! pip install ipython-sql
#! pip install mysqlclient
#!pip install ipython-sql mysql-connector-python


### Importing required libraries

In [2]:
import pymysql as sql
import pandas as pd
import mysql.connector
#import sqlalchemy
#sqlalchemy.__version__
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Loading sql magic command module
%load_ext sql


In [4]:
#Connection to the mysql database
db_name = "hrdb"
db_host = "localhost"
db_username = "root"
db_password = "2022@2024"


db_con = sql.connect(host = db_host,
                           port = int(3306),
                           user = "root",
                           password = db_password,
                           db = db_name)
db_con    

In [5]:
mycursor = db_con.cursor()
mycursor

In [6]:
#Showing database tables
tables = "SHOW Tables;"
tables_ = pd.read_sql(tables, db_con)
tables_

,Tables_in_hrdb
0,departments
1,employees
2,job_history
3,jobs
4,locations
5,petrescue


In [7]:
#Displaying the employees table data
query1 = ("SELECT * FROM employees")

emp_data = pd.read_sql_query(query1, db_con, index_col=None)
emp_data.head() 

,EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
0,E1001,John,Thomas,123456,1976-09-01,M,"5631 Rice, OakPark,IL",100,100000.0,30001,2
1,E1002,Alice,James,123457,1972-07-31,F,"980 Berry ln, Elgin,IL",200,80000.0,30002,5
2,E1003,Steve,Wells,123458,1980-10-08,M,"291 Springs, Gary,IL",300,50000.0,30002,5
3,E1004,Santosh,Kumar,123459,1985-07-20,M,"511 Aurora Av, Aurora,IL",400,60000.0,30002,5
4,E1005,Ahmed,Hussain,123410,1981-04-01,M,"216 Oak Tree, Geneva,IL",500,70000.0,30001,2


In [8]:
#Displaying department data
query2 = ("SELECT * FROM departments")

d_data = pd.read_sql(query2, db_con)
d_data.head()

,DEPT_ID_DEP,DEP_NAME,MANAGER_ID,LOC_ID
0,2,Architect Group,30001,L0001
1,5,Software Group,30002,L0002
2,7,Design Team,30003,L0003


## Querrying the database 

 ### 1 Retrieve the list of all employees, first and last names, whose first names start with ‘S’

In [9]:

emp_query1= """SELECT F_NAME, L_NAME
FROM employees
WHERE F_NAME LIKE'S%';"""

emp_names_s = pd.read_sql(emp_query1, db_con)
emp_names_s.head()

,F_NAME,L_NAME
0,Steve,Wells
1,Santosh,Kumar


### 2 Arrange all the records of the EMPLOYEES table in ascending order by the date of birth.

In [10]:

emp_query2 = """SELECT * 
FROM employees
ORDER BY B_DATE;"""

emp_ordered = pd.read_sql(emp_query2,db_con)
emp_ordered.head()

,EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
0,E1002,Alice,James,123457,1972-07-31,F,"980 Berry ln, Elgin,IL",200,80000.0,30002,5
1,E1007,Mary,Thomas,123412,1975-05-05,F,"100 Rose Pl, Gary,IL",650,65000.0,30003,7
2,E1001,John,Thomas,123456,1976-09-01,M,"5631 Rice, OakPark,IL",100,100000.0,30001,2
3,E1006,Nancy,Allen,123411,1978-06-02,F,"111 Green Pl, Elgin,IL",600,90000.0,30001,2
4,E1003,Steve,Wells,123458,1980-10-08,M,"291 Springs, Gary,IL",300,50000.0,30002,5


**3 Group the records in terms of the department IDs, filter the ones that have
 average salary more than or equal to 60000.Show department ID, department name and the average salary.
 Round the result to 2 decimal place,sort the results for each group in descending order of average salary** 
 
 

In [11]:
emp_query3 = """SELECT DEP_ID, ROUND(AVG(SALARY), 2) AS AVG_SALARY, DEP_NAME
 FROM employees AS e
 INNER JOIN departments AS d
     ON e.DEP_ID = d.DEPT_ID_DEP
 GROUP BY DEP_ID
 HAVING AVG_SALARY >= 60000
 ORDER BY AVG_SALARY DESC;"""

Grp_result = pd.read_sql(emp_query3, db_con)
Grp_result.head()

,DEP_ID,AVG_SALARY,DEP_NAME
0,2,86666.67,Architect Group
1,7,66666.67,Design Team
2,5,65000.00,Software Group


### 4 Retrieve the names of employees along with their department names.

In [12]:
emp_query4 = """SELECT F_NAME, L_NAME, DEP_NAME
FROM  employees AS e
INNER JOIN  departments AS d
ON e.DEP_ID = d.DEPT_ID_DEP;"""

emp_dep = pd.read_sql(emp_query4, db_con)
emp_dep.head()

,F_NAME,L_NAME,DEP_NAME
0,John,Thomas,Architect Group
1,Alice,James,Software Group
2,Steve,Wells,Software Group
3,Santosh,Kumar,Software Group
4,Ahmed,Hussain,Architect Group


### 5 Count how many employees hold each job title.

In [13]:
emp_query5 = """SELECT JOB_TITLE, COUNT(JOB_TITLE) AS NUM_OF_EMPLOYEES
FROM employees AS e
LEFT JOIN jobs AS j
ON  e.JOB_ID= j.JOB_IDENT
GROUP BY JOB_TITLE
ORDER BY JOB_TITLE;"""

emp_title_count = pd.read_sql(emp_query5, db_con)
emp_title_count.head()

,JOB_TITLE,NUM_OF_EMPLOYEES
0,Jr. Architect,1
1,Jr. Designer,2
2,Jr.Software Developer,2
3,Lead Architect,1
4,Sr. Architect,1


### 6 Find the average salary of employees in each departments

In [14]:
emp_query6 = """SELECT ROUND(AVG(SALARY),2)AS AVG_SALARY, DEP_NAME
FROM employees AS e
LEFT JOIN departments AS d
	ON e.DEP_ID= d.DEPT_ID_DEP
GROUP BY DEP_NAME;"""

avg_dep_salary = pd.read_sql(emp_query6, db_con)
avg_dep_salary.head()

,AVG_SALARY,DEP_NAME
0,86666.67,Architect Group
1,65000.00,Software Group
2,66666.67,Design Team


### 7 Determine which department has the most employees.

In [15]:
emp_query7 = """SELECT  DEP_NAME, COUNT(DEP_ID) AS NUM_OF_EMPLOYEES
FROM employees AS e
LEFT JOIN departments AS d
	ON e.DEP_ID = d.DEPT_ID_DEP
GROUP BY DEP_NAME;"""

dep_emp_count = pd.read_sql(emp_query7, db_con)
dep_emp_count.head()

,DEP_NAME,NUM_OF_EMPLOYEES
0,Architect Group,3
1,Software Group,4
2,Design Team,3


### 8 Retrieve all employee records whose salary is lower than the average salary.

In [16]:
 emp_query8 = """SELECT * FROM employees
WHERE SALARY < (SELECT AVG(SALARY) FROM employees
ORDER BY SALARY);"""

lower_avg_salary =  pd.read_sql(emp_query8, db_con)
lower_avg_salary.head()

,EMP_ID,F_NAME,L_NAME,SSN,B_DATE,SEX,ADDRESS,JOB_ID,SALARY,MANAGER_ID,DEP_ID
0,E1003,Steve,Wells,123458,1980-10-08,M,"291 Springs, Gary,IL",300,50000.0,30002,5
1,E1004,Santosh,Kumar,123459,1985-07-20,M,"511 Aurora Av, Aurora,IL",400,60000.0,30002,5
2,E1005,Ahmed,Hussain,123410,1981-04-01,M,"216 Oak Tree, Geneva,IL",500,70000.0,30001,2
3,E1007,Mary,Thomas,123412,1975-05-05,F,"100 Rose Pl, Gary,IL",650,65000.0,30003,7
4,E1008,Bharath,Gupta,123413,1985-06-05,M,"145 Berry Ln, Naperville,IL",660,65000.0,30003,7


### 9 Extract the first and last names of the oldest employee.

In [17]:
emp_query9 = """SELECT F_NAME, L_NAME FROM employees
WHERE B_DATE = (SELECT MIN(B_DATE) FROM employees);"""

oldest_emp = pd.read_sql(emp_query9, db_con)
oldest_emp

,F_NAME,L_NAME
0,Alice,James


### Extract the first and last names of the highest paid  employee.

In [18]:

emp_query10 = """SELECT F_NAME, L_NAME,SALARY FROM employees
WHERE SALARY = (SELECT MAX(SALARY) FROM employees);"""

high_paid_emp =pd.read_sql(emp_query10, db_con)
high_paid_emp

,F_NAME,L_NAME,SALARY
0,John,Thomas,100000.0


**11 Identify the average cost of rescuing a single dog.Note that the cost per dog would be in different instances.** 


In [19]:
emp_query11 = """SELECT ROUND(AVG(COST/QUANTITY),2) AS Dog_avg_rescue FROM petrescue
WHERE ANIMAL = 'Dog';"""
cost_dog_res = pd.read_sql(emp_query11, db_con)
cost_dog_res

,Dog_avg_rescue
0,127.27


### Show the number of rescues in the 5th month

In [20]:
emp_query12 = """SELECT SUM(QUANTITY) AS NUM_OF_RESCUES 
 FROM petrescue 
 WHERE MONTH(RESCUEDATE) = '05';"""
total_res_in_5 = pd.read_sql(emp_query12, db_con)
total_res_in_5

,NUM_OF_RESCUES
0,9.0


**13 The rescue shelter is supposed to find good homes for all animals within 1 year of their rescue. 
 Write a query to show ID and the target date.**

In [21]:
emp_query13 = """SELECT ID, DATE_ADD(RESCUEDATE, INTERVAL 1 YEAR) 
 FROM petrescue;"""

res_in_ayear = pd.read_sql(emp_query13, db_con)
res_in_ayear.head()

,ID,"DATE_ADD(RESCUEDATE, INTERVAL 1 YEAR)"
0,1,2019-05-29
1,2,2019-06-01
2,3,2019-06-04
3,4,2019-06-04
4,5,2019-06-10
